In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_excel('TP_pivot_JWBLACK.xlsx')

In [3]:
#df1.info(verbose=True)

In [4]:
df2 = df1.loc[df1['Tipo de tienda'] != 'CONVENIENCIA']

In [5]:
df3 = df2[['Store Number','SEMANA','City','State','Tipo de tienda','Region','Retailer','CADENA',
'ruta','Area Manager','Area Executive','ARETE','CABECERA','EXHIBIDORES','ISLA','MAMUT','PALLET','PASTELERO','PDQ']]

In [6]:
weeks = [22]

In [7]:
weeks_headers = ['week22'] #length must be the same as previous line

In [8]:
df4 = df3[df3['SEMANA'].isin(weeks)]

In [9]:
df5 = df4.groupby(['Store Number','SEMANA','City','State','Tipo de tienda','Region','Retailer','CADENA','ruta',
'Area Manager','Area Executive']).sum().reset_index()

In [10]:
df5['Score VDM'] = df5['ARETE']*2

In [11]:
df5['Cabecera Score'] = df5['CABECERA']*4

In [12]:
df5['Exhibidores Score'] = df5['EXHIBIDORES']*5

In [13]:
df5['Isla Score'] = df5['ISLA']*3

In [14]:
df5['Mamut Score'] = df5['MAMUT']*3

In [15]:
df5['Pallet Score'] = df5['PALLET']*3

In [16]:
df5['Pastelero Score'] = df5['PASTELERO']*3

In [17]:
df5['PDQ Score'] = df5['PDQ']*3

In [18]:
df5['Total Score'] = df5['Arete Score'] + df5['Cabecera Score'] + df5['Exhibidores Score'] + df5['Isla Score'] + df5['Mamut Score'] + df5['Pallet Score'] + df5['Pastelero Score'] + df5['PDQ Score']

In [19]:
df_score = df5[['Store Number','ruta','SEMANA',
'Arete Score','Cabecera Score','Exhibidores Score','Isla Score','Mamut Score','Pallet Score','Pastelero Score','PDQ Score',
'Total Score']]

In [20]:
df_score_ruta = df_score.groupby(['ruta','SEMANA']).agg({
'Store Number': 'count',
'Arete Score': 'sum',
'Cabecera Score': 'sum',
'Exhibidores Score': 'sum',
'Isla Score': 'sum',
'Mamut Score': 'sum',
'Pallet Score': 'sum',
'Pastelero Score': 'sum',
'PDQ Score': 'sum',
'Total Score': 'sum'
}).reset_index()

In [21]:
df_score_ruta = df_score_ruta[['ruta','SEMANA','Total Score']]

In [22]:
pivot_weekly_score = pd.pivot_table(df_score_ruta,
index=['ruta'],
columns=['SEMANA'], aggfunc='sum', fill_value=0).reset_index()

In [23]:
pivot_weekly_score.columns = pivot_weekly_score.columns.get_level_values(0)

In [24]:
pivot_weekly_score.columns = pivot_weekly_score.columns[:1].tolist() + weeks_headers

In [25]:
df_leaderboard = pivot_weekly_score

In [26]:
df_leaderboard['Final Score'] = df_leaderboard.sum(axis=1, numeric_only=True)

In [27]:
df_leaderboard = df_leaderboard.sort_values(by='Final Score', ascending=False)

In [28]:
df_leaderboard['Lugar'] = np.arange(1,len(df_leaderboard)+1)

In [29]:
df_leaderboard.head() #portada del sitio web

,ruta,week22,Final Score,Lugar
265,PROM.SEM.98,87,87,1
264,PROM.SEM.97,86,86,2
164,PROM.SEM.305,75,75,3
186,PROM.SEM.325,67,67,4
36,PROM.SEM.143,62,62,5


In [30]:
df_score.head()

,Store Number,ruta,SEMANA,Arete Score,Cabecera Score,Exhibidores Score,Isla Score,Mamut Score,Pallet Score,Pastelero Score,PDQ Score,Total Score
0,29,PROM.SEM.34,22,0,0,0,0,0,3,0,0,3
1,30,PROM.SEM.145,22,0,0,0,0,0,9,0,0,9
2,33,PROM.SEM.313,22,0,0,0,0,0,3,0,0,3
3,36,PROM.SEM.332,22,0,0,0,0,0,3,0,0,3
4,37,PROM.SEM.65,22,0,0,0,0,0,3,0,0,3


In [33]:
df_leaderboard = df_leaderboard.rename(columns={
'Final Score': 'fs'
})

In [34]:
df_score = df_score.rename(columns={
'Store Number': 'store',
'SEMANA': 'week',
'Arete Score': 'arete',
'Cabecera Score': 'cabecera',
'Exhibidores Score': 'exhibidores',
'Isla Score': 'isla',
'Mamut Score': 'mamut',
'Pallet Score': 'pallet',
'Pastelero Score': 'pastelero',
'PDQ Score': 'pdq',
'Total Score': 'score'
})

In [35]:
df2_store_names = df2[['Store Number','Store Name']]

In [36]:
df2_store_names = df2_store_names.rename(columns={
'Store Number': 'store',
'Store Name': 'name'
})

In [37]:
df2_store_names = df2_store_names.drop_duplicates(subset='store')

In [38]:
df_score = df_score.merge(df2_store_names, how='left')

In [40]:
df_score = df_score[['name','ruta','week','arete','cabecera','exhibidores','isla','mamut','pallet','pastelero','pdq','score']]

In [ ]:
#aquí puedes agregar pasos para consolidar, o no jeje

In [41]:
df_leaderboard.to_json('leaderboard.js', orient='records')

In [42]:
df_score.to_json('score.js', orient='records')